In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

In [ ]:
np.random.seed(171)

# 事前準備

講義で使うための補助関数を定義しておきます。(この関数を理解する必要はありません。)

In [ ]:
import pydot
from IPython import display
from sklearn import tree
from io import StringIO

In [ ]:
def draw_tree(model):
    data = StringIO()
    tree.export_graphviz(model, out_file=data)
    graphs = pydot.graph_from_dot_data(data.getvalue())
    for graph in graphs:
        display.display(display.Image(graph.create_png()))

# データのロード

演習で使うデータをロードします。  

データは https://www.kaggle.com/uciml/human-activity-recognition-with-smartphones から取得したものです。  

取得時のライセンスはCC0: Public Domainです。

In [ ]:
train = pd.read_csv("data/train.csv")
dev = pd.read_csv("data/dev.csv")
test = pd.read_csv("data/test.csv")

# データの確認

In [ ]:
train.head()

カラム名だけでも全部表示して確認

In [ ]:
print(train.columns.tolist())

subjectの中身を確認(配布元によるとユーザーIDのようなものとのこと)

In [ ]:
train["subject"].unique()

分類対象となるクラスを確認

In [ ]:
train["Activity"].unique()

# データ整形

正解データを別変数に分離

In [ ]:
train_t = train["Activity"]
train = train.drop("Activity", axis=1)

In [ ]:
dev_t = dev["Activity"]
dev = dev.drop("Activity", axis=1)

In [ ]:
test_t = test["Activity"]
test = test.drop("Activity", axis=1)

subjectも使いたいところだが、カテゴリカル変数だと具合が悪いので今回はカット

In [ ]:
train = train.drop("subject", axis=1)
dev = dev.drop("subject", axis=1)
test = test.drop("subject", axis=1)

特徴量ベクトルを作る

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
def df_to_features(train_df, dev_df, test_df):
    scaler = StandardScaler()
    train_features = scaler.fit_transform(train_df)
    dev_features = scaler.transform(dev_df)
    test_features = scaler.transform(test_df)
    return train_features, dev_features, test_features

In [ ]:
train_features, dev_features, test_features = df_to_features(train, dev, test)

# 演習に必要なモジュールを読み込みます。

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# 演習1

SVMを使ってみましょう。  
Scikit Learnでは、SVCを使います。

ドキュメント: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

学習しましょう。

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = #TODO ここでSVCを使ってください。

In [ ]:
svm.fit(train, train_t)

評価しましょう。

In [ ]:
pred_svm = svm.predict(test)

In [ ]:
pd.crosstab(test_t, pred_svm)

In [ ]:
print(classification_report(test_t, pred_svm))

# 演習2

決定木とRandomForestを使ってみましょう。

## 決定木

決定木にはDecisionTreeClassifierを使います。  

ドキュメント: http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

学習しましょう。

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = # TODO ここでDecisionTreeClassifierを使ってください。

In [ ]:
dt.fit(train, train_t)

評価しましょう。

In [ ]:
pred_dt = dt.predict(test)

In [ ]:
pd.crosstab(test_t, pred_dt)

In [ ]:
print(classification_report(test_t, pred_dt))

どんな木ができたか見てみましょう。
draw_tree()は、今回の講義のために作成した関数です。  
モデルを引数に渡すとグラフを描画します。

In [ ]:
draw_tree(dt)

## RandomForest

RandomForestを使うには、RandomForestClassifierを使います。

ドキュメント: http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

学習しましょう。

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = # TODO ここでRandomForestClassifierを使ってください。

In [ ]:
rf.fit(train, train_t)

評価しましょう。

In [ ]:
# TODO テストデータを使って評価してください。(rf.predict)

In [ ]:
# TODO Confusion Matrixを作ってください。(pd.crosstab)

In [ ]:
# TODO Precision, Recall, F値を表示してください。

# 演習3

SVMをチューニングしてみましょう。

チューニング用データにおいて、F値を最大とするパラメータを探します。

In [ ]:
f1_scores = []
for c in [1, 10, 20, 50, 100, 200, 500, 1000]:
    # カーネルは線形(linear), RBF, 多項式(poly)を試します。
    for kernel in ["linear", "rbf", "poly"]:
        # モデルを学習します。
        svm = # TODO パラメータCとkernelをセットしつつSVCを作ってください。
        svm.fit(train, train_t)
        
        # チューニング用データの性能を測定します。
        pred_svm_dev = svm.predict(dev)
        score = f1_score(dev_t, pred_svm_dev, average="macro")
        
        # 結果を表示、保存します。
        print("C={} kernel={} score={}".format(c, kernel, score))
        f1_scores.append((c, kernel, score, svm))

チューニング結果のグラフを確認しましょう。

In [ ]:
cs, kernels, scores, _ = list(zip(*f1_scores))
tuning_result = pd.DataFrame(dict(C=cs, kernel=kernels, score=scores))

In [ ]:
tuning_result.head()

In [ ]:
piv = tuning_result.pivot_table(index="C", columns="kernel", values="score")

In [ ]:
piv.plot(kind="line", logx=True)

スコア最大となるパラメータの組み合わせを表示しましょう。

In [ ]:
mc, mk, ms, msvm = max(f1_scores, key=lambda x: x[2])
print("C={}, kernel={} のとき、チューニング用データのスコアは最大値 {}".format(mc, mk, ms))

最終性能を求めましょう。  
上記、msvmにチューニングデータで最大性能となるモデルが格納されているので、それをテストデータに適用します。

In [ ]:
# TODO テストデータに対する予測を実行してください。

In [ ]:
# TODO Confusion Matrixを作成してください。

In [ ]:
# TODO Precision, Recall, F値を表示してください。